## 2.  핫플지수 확정



In [8]:
import polars as pl
import numpy as np

print("✅ 1️⃣ Parquet 스캔 시작")
lazy_df = pl.scan_parquet('combined_parquet/**/*.parquet', glob=True)
print("   → LazyFrame 생성 완료")


✅ 1️⃣ Parquet 스캔 시작
   → LazyFrame 생성 완료


In [9]:
import polars as pl

# ———————— 풀 테이블 출력 설정 ————————
pl.Config.set_tbl_rows(100)         # 최대 100행까지 줄임표 없이 표시
pl.Config.set_tbl_cols(100)        # 최대 100열까지 표시
pl.Config.set_tbl_formatting("UTF8_FULL")  # 테두리 포함, 생략 없이 전체 보여주기


polars.config.Config

In [10]:

import polars as pl


# 1️⃣ filtered 정의: 2024년··주요 목적 필터 + 날짜 파싱
filtered = (
    lazy_df
      # ─ 날짜 범위: 2024-01-01 ~ 2024-12-31
      .filter((pl.col("ETL_YMD") >= "20240101") & (pl.col("ETL_YMD") <= "20241231"))
      
    #   # ─ 시간 범위: 18시(18)부터 23시(23)까지
    #   .filter((pl.col("TIME_CD").cast(pl.Int64) >= 18) 
    #         & (pl.col("TIME_CD").cast(pl.Int64) <= 23))
      
    #   # ─ 목적 필터: 관심 목적군만
    #   .filter(pl.col("MOVE_PURPOSE").is_in(
    #       ["쇼핑","관광","출근","귀가","병원","기타","등교"]
    #   ))


      # ─ ETL_YMD 문자열을 Date 타입으로 바꿔서
      .with_columns(
          pl.col("ETL_YMD")
            .str.strptime(pl.Date, format="%Y%m%d")
            .alias("ETL_DATE")
      )

#       # ─ 평일(월~금)만 남기기 (weekday 0~4)
#       .filter(pl.col("ETL_DATE").dt.weekday().is_in([0,1,2,3,4]))
      
)

# 이제 `filtered`에 “원하는 기간·시간·목적·요일” 이 모두 적용되어 있으니,
# 아래처럼 곧장 동별 연간 통계 계산에 쓰실 수 있습니다.



In [13]:
import polars as pl

# 1️⃣ STEP1: 2024년·주요 목적 필터
step1 = (
    lazy_df
      .filter((pl.col('ETL_YMD') >= '20240101') & (pl.col('ETL_YMD') <= '20241231'))
      .filter(pl.col('MOVE_PURPOSE').is_in(['쇼핑','관광','출근','귀가','병원','기타']))
)

# 2️⃣ STEP2: 날짜 파싱 + 주말(토·일)만 걸러내기
step2 = (
    step1
      .with_columns(
         pl.col('ETL_YMD').str.strptime(pl.Date, '%Y%m%d').alias('ETL_DATE')
      )
      .filter(  # 여기에만 주말 필터
         pl.col('ETL_DATE').dt.weekday().is_in([5, 6])
      )
)

# 3️⃣ STEP3: 2030 합계 · 전체 합계 · 2030 비율
young_cols = ['MALE_20_CNT','MALE_30_CNT','FEML_20_CNT','FEML_30_CNT']
all_cols   = young_cols + [ 'MALE_00_CNT','MALE_10_CNT','MALE_40_CNT','MALE_50_CNT',
                            'MALE_60_CNT','MALE_70_CNT','FEML_00_CNT','FEML_10_CNT',
                            'FEML_40_CNT','FEML_50_CNT','FEML_60_CNT','FEML_70_CNT' ]

young_sum = sum((pl.col(c).cast(pl.Float64) for c in young_cols), pl.lit(0.0)).alias('YOUNG_VISITORS')
total_sum = sum((pl.col(c).cast(pl.Float64) for c in all_cols),   pl.lit(0.0)).alias('TOTAL_VISITORS')

step3 = step2.with_columns([
    young_sum,
    total_sum,
    (young_sum / (total_sum + 1e-6)).alias('YOUNG_RATIO'),
])

print("▶ step3 샘플 (주말만):")
print(
    step3
      .select(['ETL_DATE','FULL_NM','YOUNG_RATIO','YOUNG_VISITORS','TOTAL_VISITORS'])
      .limit(10).collect()
)


▶ step3 샘플 (주말만):
shape: (10, 5)
┌────────────┬──────────────────────────────┬─────────────┬────────────────┬────────────────┐
│ ETL_DATE   ┆ FULL_NM                      ┆ YOUNG_RATIO ┆ YOUNG_VISITORS ┆ TOTAL_VISITORS │
│ ---        ┆ ---                          ┆ ---         ┆ ---            ┆ ---            │
│ date       ┆ str                          ┆ f64         ┆ f64            ┆ f64            │
╞════════════╪══════════════════════════════╪═════════════╪════════════════╪════════════════╡
│ 2024-10-04 ┆ 경기도_안산시_상록구_해양동  ┆ 0.210304    ┆ 26.41          ┆ 125.58         │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2024-10-04 ┆ 경기도_광명시_일직동         ┆ 0.500771    ┆ 438.18         ┆ 875.01         │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2024-10-04 ┆ 서울특별시_노원구_상계8동    ┆ 0.205512    ┆ 55.93          ┆ 272.15         │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼

In [14]:
# ─── Hotness3 계산 & 동별 연간 TOP30 ────────────────────────────
import polars as pl

# 1️⃣ 행(row) 단위 20·30대 여성 합계 Expr
female2030 = (
    pl.col("FEML_20_CNT").cast(pl.Float64)
  + pl.col("FEML_30_CNT").cast(pl.Float64)
).alias("ROW_FEML2030")

# 2️⃣ step3 에 추가 (step3에는 이미 “주말만” 필터, YOUNG_VISITORS·TOTAL_VISITORS 계산 완료)
step3_mod = step3.with_columns([ female2030 ])

# 3️⃣ 동별 연간 합산 및 Hotness3 계산
top30_h3 = (
    step3_mod
      .group_by("FULL_NM")
      .agg([
        # 1년치 20·30대 여성 방문자 합
        pl.sum("ROW_FEML2030").alias("ANNUAL_FEML2030"),
        # 1년치 주말 전체 방문자 합 (step3 only contains weekend rows)
        pl.sum("TOTAL_VISITORS").alias("ANNUAL_WEEKEND_TOTAL")
      ])
      .with_columns([
        # Hotness3 = √(ANNUAL_FEML2030) / √(ANNUAL_WEEKEND_TOTAL)
        (
          pl.col("ANNUAL_FEML2030").sqrt()
          / pl.col("ANNUAL_WEEKEND_TOTAL").sqrt()
        ).alias("HOTNESS3")
      ])
      .sort("HOTNESS3", descending=True)
      .limit(30)
      .collect()
)

print("✅ 동별 연간 Hotness3 TOP30:")
print(top30_h3.select(["FULL_NM","HOTNESS3"]))


✅ 동별 연간 Hotness3 TOP30:
shape: (30, 2)
┌──────────────────────────────┬──────────┐
│ FULL_NM                      ┆ HOTNESS3 │
│ ---                          ┆ ---      │
│ str                          ┆ f64      │
╞══════════════════════════════╪══════════╡
│ 서울특별시_마포구_연남동     ┆ 0.60645  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_마포구_서교동     ┆ 0.591466 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_마포구_합정동     ┆ 0.563482 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_성동구_성수1가2동 ┆ 0.556252 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_광진구_화양동     ┆ 0.550003 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_성동구_성수2가3동 ┆ 0.543344 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_서대문구_신촌동   ┆ 0.540416 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_용산구_한남동     ┆ 0.540279 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_용산구_용산2가동  ┆ 0.539612 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_용산구_이태원1동  ┆ 0.53932  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌

In [15]:
# ── 5) 상위30 평균·중앙값 ──
mean_h2   = top30_h3["HOTNESS3"].mean()
median_h2 = top30_h3["HOTNESS3"].median()
print(f"\n▶ HOTNESS3 상위30 평균 : {mean_h2:.6f}")
print(f"▶ HOTNESS3 상위30 중앙값: {median_h2:.6f}")


▶ HOTNESS3 상위30 평균 : 0.529685
▶ HOTNESS3 상위30 중앙값: 0.526320


In [16]:
# ─── 관악구 난향동 Hotness3 값 추출 ────────────────────────────

# 1️⃣ step3_mod / annual3_h3 같은 중간 변수를 이미 갖고 계시다면,
#    그걸 그대로 사용하시면 됩니다. 만약 바로 사용하려면 아래처럼 한 번에:

nanhyang_h3 = (
    step3_mod
      .group_by("FULL_NM")
      .agg([
        pl.sum("ROW_FEML2030").alias("ANNUAL_FEML2030"),
        pl.sum("TOTAL_VISITORS").alias("ANNUAL_WEEKEND_TOTAL")
      ])
      .with_columns([
        (
          pl.col("ANNUAL_FEML2030").sqrt()
          / pl.col("ANNUAL_WEEKEND_TOTAL").sqrt()
        ).alias("HOTNESS3")
      ])
      .filter(pl.col("FULL_NM") == "서울특별시_관악구_난향동")
      .select(["FULL_NM","HOTNESS3"])
      .collect()
)

print("▶ 난향동 연간 Hotness3:")
print(nanhyang_h3)


▶ 난향동 연간 Hotness3:
shape: (1, 2)
┌──────────────────────────┬──────────┐
│ FULL_NM                  ┆ HOTNESS3 │
│ ---                      ┆ ---      │
│ str                      ┆ f64      │
╞══════════════════════════╪══════════╡
│ 서울특별시_관악구_난향동 ┆ 0.327909 │
└──────────────────────────┴──────────┘


In [18]:
# 1️⃣ “20·30대 여성” 합계 Expr 정의 (주말만 남긴 step3 기준)
female2030 = (
    pl.col("FEML_20_CNT").cast(pl.Float64)
  + pl.col("FEML_30_CNT").cast(pl.Float64)
).alias("ROW_FEML2030")

# 2️⃣ step3에 붙이기 (step3은 이미 ‘2024년·주말’ + YOUNG_VISITORS·TOTAL_VISITORS 계산 완료)
step3_mod = step3.with_columns([ female2030 ])

# 3️⃣ 자치구(SGG_NM)별 연간 합산 & Hotness3 계산
district_h3 = (
    step3_mod
      .group_by("SGG_NM")
      .agg([
        pl.sum("ROW_FEML2030").alias("ANNUAL_FEML2030"),       # 분자
        pl.sum("TOTAL_VISITORS").alias("ANNUAL_WEEKEND_TOTAL") # 분모
      ])
      .with_columns([
        # √(여성2030합) / √(주말전체합)
        (
          pl.col("ANNUAL_FEML2030").sqrt()
          / pl.col("ANNUAL_WEEKEND_TOTAL").sqrt()
        ).alias("HOTNESS3")
      ])
      .sort("HOTNESS3", descending=True)  # 하위(낮은 값)부터 정렬
      .collect()
)

print("▶ 자치구별 연간 Hotness3 (높은 순):")
print(district_h3.select(["SGG_NM","HOTNESS3"]))


▶ 자치구별 연간 Hotness3 (높은 순):
shape: (78, 2)
┌─────────────────┬──────────┐
│ SGG_NM          ┆ HOTNESS3 │
│ ---             ┆ ---      │
│ str             ┆ f64      │
╞═════════════════╪══════════╡
│ 마포구          ┆ 0.505553 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 용산구          ┆ 0.491936 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 강남구          ┆ 0.462547 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 영등포구        ┆ 0.456979 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 종로구          ┆ 0.456746 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 성동구          ┆ 0.454413 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 관악구          ┆ 0.439411 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 광진구          ┆ 0.435916 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 수원시 팔달구   ┆ 0.435616 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서대문구        ┆ 0.434955 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 중구            ┆ 0.431652 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 송파구          ┆ 0.428555 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 서초구          ┆ 0.427383 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ 강서구          ┆ 0.421254 │
├╌╌╌╌╌╌╌╌╌╌╌